### Notebook for cleaning the Leetcode Question dataset
- No void functions
- No class implementations
- No external definitions
- No examples in description


In [2]:
import pandas as pd
import re
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
from leetcode_env.utils import PySubmissionFormatter, RsSubmissionFormatter

In [8]:
data = pd.read_csv('data/with_snippets/leetcode_hard_with_snippets_uncontaminated.csv')

### Remove rows that require external definitions

In [9]:
no_defs_inds = [ind for ind, row in data.iterrows() if row['cpp_snippet'].split(' ')[0] == 'class']
no_defs = data.iloc[no_defs_inds]

### Remove rows that return none

In [10]:
ret_inds = [ind for ind, row in no_defs.iterrows() if '\"\"\"' in row['python3_snippet'].split('\n')[2]]
ret = no_defs.drop(ret_inds)

### Remove rows that require class implementation

In [11]:
function_name_regex = r"(?<=def\s)\w+"
impl_inds = [ind for ind, row in no_defs.iterrows()
             if re.search(function_name_regex, row['python3_snippet']).group(0) == '__init__']
no_impl = ret.drop(impl_inds)

### Remove Examples from descriptions

In [13]:
# delimiters = ["Example 1:",
#               "Example1:"
#               "Example:",
#               "Example :"
#               "Examples:",
#               "Valid Code Examples:",
#               "Evaluation Example:"]

def remove_examples(desc):
    lines = [l.strip() for l in desc.split('\n')]
    for i, line in enumerate(lines):
        if 'Example' in line:
            return '\n'.join(lines[:i])
    return desc

def remove_empty(desc: str):
    return '\n'.join(line for line in desc.split('\n') if line.strip())

for ind, row in no_impl.iterrows():
    res = remove_empty(remove_examples(row['description']))
    no_impl.at[ind, 'description'] = res

### Save

In [14]:
no_impl.to_csv('data/with_snippets/leetcode_hard_with_snippets_uncontaminated_cleaned.csv')